# Grocery Sales Forecasting - Complete EDA & Feature Engineering

This notebook performs comprehensive data cleaning, merging, feature engineering, and exploratory data analysis for predicting 1-16 days ahead of unit_sales.

**Goal:** Prepare data for Item-Store Daily Forecasting model

**Strategy:** 
- Feature engineering on FULL 125M dataset
- Stratified sampling ONLY for visualizations (fast EDA)
- Final outputs include full engineered dataset for modeling

**Contents:**
1. Data Loading & Cleaning
2. Dataset Merging → **Save merged.csv**
3. Feature Engineering on FULL dataset (~30 features) → **Save featured.csv**
4. Stratified Sampling (2M rows for visualizations only)
5. 10 Essential Visualizations (on sample)
6. Train/Test Split (on full dataset) → **Save train.csv & test.csv**

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

Libraries imported successfully!
Pandas version: 2.3.3
NumPy version: 2.3.5


## 2. Data Loading & Initial Cleaning

In [2]:
# Load all datasets
print("Loading datasets...")

df_train = pd.read_parquet('../data/raw/df_train_all_v1.parquet')
df_items = pd.read_parquet('../data/raw/items.parquet')
df_stores = pd.read_parquet('../data/raw/stores.parquet')
df_oil = pd.read_parquet('../data/raw/oil.parquet')
df_transactions = pd.read_parquet('../data/raw/transactions.parquet')
df_holidays = pd.read_parquet('../data/raw/holiday_events.parquet')

print(f"✓ Train data: {df_train.shape}")
print(f"✓ Items: {df_items.shape}")
print(f"✓ Stores: {df_stores.shape}")
print(f"✓ Oil: {df_oil.shape}")
print(f"✓ Transactions: {df_transactions.shape}")
print(f"✓ Holidays: {df_holidays.shape}")

# Display sample data
print("\n" + "="*50)
print("SAMPLE DATA:")
print("="*50)
print("\nTrain data:")
print(df_train.head(3))
print("\nItems:")
print(df_items.head(3))
print("\nStores:")
print(df_stores.head(3))

Loading datasets...
✓ Train data: (125497040, 5)
✓ Items: (4100, 3)
✓ Stores: (54, 4)
✓ Oil: (1218, 2)
✓ Transactions: (83488, 3)
✓ Holidays: (350, 6)

SAMPLE DATA:

Train data:
        date  store_nbr  item_nbr  unit_sales  onpromotion
0 2013-01-01         25    103665    2.079442        False
1 2013-01-01         25    105574    0.693147        False
2 2013-01-01         25    105575    1.098612        False

Items:
             family  class  perishable
item_nbr                              
96995     GROCERY I   1093           0
99197     GROCERY I   1067           0
103501     CLEANING   3008           0

Stores:
            city      state type  cluster
store_nbr                                
1          Quito  Pichincha    D       13
2          Quito  Pichincha    D       13
3          Quito  Pichincha    D        8
✓ Train data: (125497040, 5)
✓ Items: (4100, 3)
✓ Stores: (54, 4)
✓ Oil: (1218, 2)
✓ Transactions: (83488, 3)
✓ Holidays: (350, 6)

SAMPLE DATA:

Train data:
      

### 2.1 Convert Date Columns to Datetime

In [3]:
print("Converting date columns to datetime...")

# Convert date columns
df_train['date'] = pd.to_datetime(df_train['date'])
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'])

print("✓ All date columns converted to datetime")
print(f"\nDate ranges:")
print(f"Train: {df_train['date'].min()} to {df_train['date'].max()}")
print(f"Oil: {df_oil['date'].min()} to {df_oil['date'].max()}")
print(f"Transactions: {df_transactions['date'].min()} to {df_transactions['date'].max()}")
print(f"Holidays: {df_holidays['date'].min()} to {df_holidays['date'].max()}")

Converting date columns to datetime...
✓ All date columns converted to datetime

Date ranges:
✓ All date columns converted to datetime

Date ranges:
Train: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
Oil: 2013-01-01 00:00:00 to 2017-08-31 00:00:00
Transactions: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
Holidays: 2012-03-02 00:00:00 to 2017-12-26 00:00:00
Train: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
Oil: 2013-01-01 00:00:00 to 2017-08-31 00:00:00
Transactions: 2013-01-01 00:00:00 to 2017-08-15 00:00:00
Holidays: 2012-03-02 00:00:00 to 2017-12-26 00:00:00


### 2.2 Interpolate Missing Oil Prices (43 values)

In [4]:
print("Handling missing oil prices...")
print(f"Missing oil prices before: {df_oil['dcoilwtico'].isna().sum()}")

# Linear interpolation
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].interpolate(method='linear')

# Forward fill for any remaining
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].fillna(method='ffill')

# Backward fill for any at the beginning
df_oil['dcoilwtico'] = df_oil['dcoilwtico'].fillna(method='bfill')

print(f"Missing oil prices after: {df_oil['dcoilwtico'].isna().sum()}")
print("✓ Oil prices interpolated successfully")

Handling missing oil prices...
Missing oil prices before: 43
Missing oil prices after: 0
✓ Oil prices interpolated successfully


## 3. Dataset Merging (Sequential Order)

In [5]:
print("Merging datasets in order...")
print(f"Starting with train data: {df_train.shape}")

# Step 1: Merge items
df = df_train.merge(df_items, on='item_nbr', how='left')
print(f"After merging items: {df.shape}")

# Step 2: Merge stores
df = df.merge(df_stores, on='store_nbr', how='left')
print(f"After merging stores: {df.shape}")

# Step 3: Merge oil
df = df.merge(df_oil, on='date', how='left')
print(f"After merging oil: {df.shape}")

# Step 4: Merge transactions
df = df.merge(df_transactions, on=['date', 'store_nbr'], how='left')
print(f"After merging transactions: {df.shape}")

# Step 5: Merge holidays - create holiday flags
df_holidays_clean = df_holidays.copy()
df_holidays_clean['is_holiday'] = 1
# Keep only necessary columns from holidays
holidays_agg = df_holidays_clean.groupby('date').agg({
    'is_holiday': 'max',
    'type': lambda x: ', '.join(x.unique()),
    'transferred': 'max'
}).reset_index()
holidays_agg.columns = ['date', 'is_holiday', 'holiday_type', 'holiday_transferred']

df = df.merge(holidays_agg, on='date', how='left')
df['is_holiday'] = df['is_holiday'].fillna(0).astype(int)
print(f"After merging holidays: {df.shape}")

print(f"\n✓ Final merged dataset: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

Merging datasets in order...
Starting with train data: (125497040, 5)
After merging items: (125497040, 8)
After merging items: (125497040, 8)
After merging stores: (125497040, 12)
After merging stores: (125497040, 12)
After merging oil: (125497040, 13)
After merging oil: (125497040, 13)
After merging transactions: (125497040, 14)
After merging transactions: (125497040, 14)
After merging holidays: (125497040, 17)

✓ Final merged dataset: (125497040, 17)

Columns: ['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family', 'class', 'perishable', 'city', 'state', 'type', 'cluster', 'dcoilwtico', 'transactions', 'is_holiday', 'holiday_type', 'holiday_transferred']
After merging holidays: (125497040, 17)

✓ Final merged dataset: (125497040, 17)

Columns: ['date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion', 'family', 'class', 'perishable', 'city', 'state', 'type', 'cluster', 'dcoilwtico', 'transactions', 'is_holiday', 'holiday_type', 'holiday_transferred']


### 3.1 Save Merged Dataset

In [6]:
print("Saving merged dataset...")

# Save merged dataset (full 125M rows)
output_path = '../results/df_merged.csv'
df.to_csv(output_path, index=False)

print(f"✓ Merged dataset saved to: {output_path}")
print(f"  Shape: {df.shape}")
print(f"  Size on disk: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB in memory")

Saving merged dataset...
✓ Merged dataset saved to: ../results/df_merged.csv
  Shape: (125497040, 17)
  Size on disk: 48326.7 MB in memory
✓ Merged dataset saved to: ../results/df_merged.csv
  Shape: (125497040, 17)
  Size on disk: 48326.7 MB in memory


## 4. Feature Engineering on FULL Dataset (~30 Features)

**IMPORTANT:** All feature engineering is performed on the full 125M row dataset to ensure production-ready features.

In [7]:
print("Adding year column for feature engineering...")

# Add year column (needed for temporal features)
df['year'] = df['date'].dt.year

print(f"✓ Year column added")
print(f"✓ Working with FULL dataset: {df.shape}")

Adding year column for feature engineering...
✓ Year column added
✓ Working with FULL dataset: (125497040, 18)
✓ Year column added
✓ Working with FULL dataset: (125497040, 18)


### 4.1 Temporal Features (8 features)

In [8]:
print("Creating temporal features...")

# Temporal features
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_month'] = df['date'].dt.day
df['week_of_year'] = df['date'].dt.isocalendar().week
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)

temporal_features = ['year', 'month', 'day_of_week', 'day_of_month', 'week_of_year', 
                     'is_weekend', 'is_month_start', 'is_month_end']

print(f"✓ Created {len(temporal_features)} temporal features")
print(f"Features: {temporal_features}")

Creating temporal features...
✓ Created 8 temporal features
Features: ['year', 'month', 'day_of_week', 'day_of_month', 'week_of_year', 'is_weekend', 'is_month_start', 'is_month_end']
✓ Created 8 temporal features
Features: ['year', 'month', 'day_of_week', 'day_of_month', 'week_of_year', 'is_weekend', 'is_month_start', 'is_month_end']


### 4.2 Lag Features (4 features)
Creating lags grouped by (store_nbr, item_nbr) to prevent data leakage.

In [9]:
print("Creating lag features (grouped by store_nbr, item_nbr)...")

# Sort by store, item, and date
df = df.sort_values(['store_nbr', 'item_nbr', 'date']).reset_index(drop=True)

# Create lag features
df['sales_lag_1'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(1)
df['sales_lag_7'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(7)
df['sales_lag_14'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(14)
df['sales_lag_28'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].shift(28)

lag_features = ['sales_lag_1', 'sales_lag_7', 'sales_lag_14', 'sales_lag_28']

print(f"✓ Created {len(lag_features)} lag features")
print(f"Features: {lag_features}")
print(f"Missing values in lag_1 (expected for first days): {df['sales_lag_1'].isna().sum():,}")

Creating lag features (grouped by store_nbr, item_nbr)...


MemoryError: Unable to allocate 1.87 GiB for an array with shape (2, 125497040) and data type object

### 4.3 Rolling Statistics (6 features)
Creating rolling windows grouped by (store_nbr, item_nbr).

In [ ]:
print("Creating rolling statistics features...")

# Rolling windows (using shift to prevent data leakage)
df['rolling_mean_7'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).mean()
)
df['rolling_mean_14'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: x.shift(1).rolling(window=14, min_periods=1).mean()
)
df['rolling_mean_28'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: x.shift(1).rolling(window=28, min_periods=1).mean()
)
df['rolling_std_7'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).std()
)
df['rolling_max_7'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).max()
)
df['rolling_min_7'] = df.groupby(['store_nbr', 'item_nbr'])['unit_sales'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).min()
)

rolling_features = ['rolling_mean_7', 'rolling_mean_14', 'rolling_mean_28', 
                    'rolling_std_7', 'rolling_max_7', 'rolling_min_7']

print(f"✓ Created {len(rolling_features)} rolling features")
print(f"Features: {rolling_features}")

### 4.4 Holiday Features (3 features)

In [ ]:
print("Creating holiday features...")

# Get unique holiday dates
holiday_dates = set(df[df['is_holiday'] == 1]['date'].unique())

# Days to next holiday
def days_to_next_holiday(date):
    future_holidays = [h for h in holiday_dates if h > date]
    if future_holidays:
        return (min(future_holidays) - date).days
    return 999  # No upcoming holiday

df['days_to_next_holiday'] = df['date'].apply(days_to_next_holiday)

# Is before holiday (3 days before)
df['is_before_holiday'] = (df['days_to_next_holiday'] <= 3).astype(int)

holiday_features = ['is_holiday', 'days_to_next_holiday', 'is_before_holiday']

print(f"✓ Created {len(holiday_features)} holiday features")
print(f"Features: {holiday_features}")
print(f"Total holidays: {df['is_holiday'].sum():,}")
print(f"Days before holidays: {df['is_before_holiday'].sum():,}")

### 4.5 Promotion Features (3 features)

In [ ]:
print("Creating promotion features...")

# Ensure onpromotion is binary
df['onpromotion'] = df['onpromotion'].fillna(0).astype(int)

# Promo lag (was there a promotion 7 days ago?)
df['promo_lag_7'] = df.groupby(['store_nbr', 'item_nbr'])['onpromotion'].shift(7)

# Days since last promotion
df_temp = df[df['onpromotion'] == 1][['store_nbr', 'item_nbr', 'date']].copy()
df_temp = df_temp.rename(columns={'date': 'last_promo_date'})

df = df.merge(
    df_temp.groupby(['store_nbr', 'item_nbr'])['last_promo_date'].max().reset_index(),
    on=['store_nbr', 'item_nbr'],
    how='left'
)

df['days_since_promo'] = (df['date'] - df['last_promo_date']).dt.days
df['days_since_promo'] = df['days_since_promo'].fillna(999)  # Never had promotion

# Promotion frequency in last 30 days
df['promo_frequency_30'] = df.groupby(['store_nbr', 'item_nbr'])['onpromotion'].transform(
    lambda x: x.shift(1).rolling(window=30, min_periods=1).sum()
)

# Clean up
df = df.drop('last_promo_date', axis=1)

promo_features = ['promo_lag_7', 'days_since_promo', 'promo_frequency_30']

print(f"✓ Created {len(promo_features)} promotion features")
print(f"Features: {promo_features}")
print(f"Items on promotion: {df['onpromotion'].sum():,} ({df['onpromotion'].mean()*100:.1f}%)")

### 4.6 Aggregate Features (4 features)

In [ ]:
print("Creating aggregate features...")

# Store daily sales (total sales across all items in that store on that day)
store_daily = df.groupby(['store_nbr', 'date'])['unit_sales'].sum().reset_index()
store_daily.columns = ['store_nbr', 'date', 'store_daily_sales']
df = df.merge(store_daily, on=['store_nbr', 'date'], how='left')

# Item daily sales (total sales of that item across all stores on that day)
item_daily = df.groupby(['item_nbr', 'date'])['unit_sales'].sum().reset_index()
item_daily.columns = ['item_nbr', 'date', 'item_daily_sales']
df = df.merge(item_daily, on=['item_nbr', 'date'], how='left')

# Family average sales (using expanding window to prevent leakage)
df = df.sort_values(['family', 'date']).reset_index(drop=True)
df['family_avg_sales'] = df.groupby('family')['unit_sales'].transform(
    lambda x: x.shift(1).expanding().mean()
)

# Store-family average sales
df = df.sort_values(['store_nbr', 'family', 'date']).reset_index(drop=True)
df['store_family_avg_sales'] = df.groupby(['store_nbr', 'family'])['unit_sales'].transform(
    lambda x: x.shift(1).expanding().mean()
)

aggregate_features = ['store_daily_sales', 'item_daily_sales', 'family_avg_sales', 'store_family_avg_sales']

print(f"✓ Created {len(aggregate_features)} aggregate features")
print(f"Features: {aggregate_features}")

### 4.7 Interaction Features (3 features)

In [ ]:
print("Creating interaction features...")

# Promotion on weekend
df['promo_weekend'] = df['onpromotion'] * df['is_weekend']

# Perishable on weekend
df['perishable_weekend'] = df['perishable'] * df['is_weekend']

# Holiday with promotion
df['holiday_promo'] = df['is_holiday'] * df['onpromotion']

interaction_features = ['promo_weekend', 'perishable_weekend', 'holiday_promo']

print(f"✓ Created {len(interaction_features)} interaction features")
print(f"Features: {interaction_features}")

### 4.8 Feature Engineering Summary & Save

In [ ]:
print("="*60)
print("FEATURE ENGINEERING SUMMARY")
print("="*60)

all_engineered_features = (temporal_features + lag_features + rolling_features + 
                           holiday_features + promo_features + aggregate_features + 
                           interaction_features)

print(f"\nTotal engineered features: {len(all_engineered_features)}")
print(f"\nBreakdown:")
print(f"  - Temporal: {len(temporal_features)}")
print(f"  - Lag: {len(lag_features)}")
print(f"  - Rolling: {len(rolling_features)}")
print(f"  - Holiday: {len(holiday_features)}")
print(f"  - Promotion: {len(promo_features)}")
print(f"  - Aggregate: {len(aggregate_features)}")
print(f"  - Interaction: {len(interaction_features)}")

print(f"\nFull dataset shape: {df.shape}")
print(f"Total columns: {len(df.columns)}")

# Display sample with new features
print("\nSample of engineered features:")
feature_cols = ['date', 'store_nbr', 'item_nbr', 'unit_sales'] + all_engineered_features[:10]
print(df[feature_cols].head())

# Save full featured dataset
print("\n" + "="*60)
print("SAVING FULL FEATURED DATASET")
print("="*60)

output_path = '../results/df_featured_full.csv'
df.to_csv(output_path, index=False)

print(f"✓ Full featured dataset saved to: {output_path}")
print(f"  Shape: {df.shape}")
print(f"  Columns: {len(df.columns)}")
print(f"  Ready for modeling!")

## 5. Create Stratified Sample (2M rows - FOR VISUALIZATIONS ONLY)

In [ ]:
print("Creating stratified sample for VISUALIZATIONS ONLY...")
print("(Full dataset preserved for final modeling)")

# Create stratification key
df['strata'] = df['year'].astype(str) + '_' + df['family'] + '_' + df['type']

# Calculate sample size per stratum
n_sample = 2_000_000
total_rows = len(df)
sample_fraction = n_sample / total_rows

print(f"\nFull dataset: {total_rows:,} rows")
print(f"Sample size: {n_sample:,} rows")
print(f"Sample fraction: {sample_fraction:.4f}")

# Stratified sampling
df_sample = df.groupby('strata', group_keys=False).apply(
    lambda x: x.sample(frac=sample_fraction, random_state=42) if len(x) > 1 else x
).reset_index(drop=True)

print(f"\n✓ Sample created: {df_sample.shape}")
print(f"\nYears distribution in sample:")
print(df_sample['year'].value_counts().sort_index())
print(f"\nFamilies in sample: {df_sample['family'].nunique()}")
print(f"Store types in sample: {df_sample['type'].nunique()}")

# Save sample for future visualization
output_path = '../results/df_sample_2m.csv'
df_sample.to_csv(output_path, index=False)
print(f"\n✓ Sample saved to: {output_path}")

print("\n" + "="*60)
print("⚠️  IMPORTANT: Sample is ONLY for visualizations below")
print("⚠️  Full dataset (df) is preserved for final train/test split")
print("="*60)

## 6. Exploratory Data Analysis - 10 Essential Visualizations (on 2M sample)

### Visualization 1: Daily Aggregate Sales Trend (2013-2017)

In [ ]:
plt.figure(figsize=(15, 6))

daily_sales = df_sample.groupby('date')['unit_sales'].sum().reset_index()
plt.plot(daily_sales['date'], daily_sales['unit_sales'], linewidth=0.8, color='steelblue')
plt.title('Daily Aggregate Sales Trend (2013-2017) - Sample Data', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Total Unit Sales (log scale)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Overall trend: Min={daily_sales['unit_sales'].min():.2f}, Max={daily_sales['unit_sales'].max():.2f}, Mean={daily_sales['unit_sales'].mean():.2f}")

### Visualization 2: Log Unit Sales Distribution

In [ ]:
plt.figure(figsize=(12, 6))

plt.hist(df_sample['unit_sales'], bins=100, edgecolor='black', alpha=0.7, color='coral')
plt.title('Distribution of Log Unit Sales', fontsize=16, fontweight='bold')
plt.xlabel('Unit Sales (log scale)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.axvline(df_sample['unit_sales'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean = {df_sample["unit_sales"].mean():.2f}')
plt.axvline(df_sample['unit_sales'].median(), color='green', linestyle='--', linewidth=2, label=f'Median = {df_sample["unit_sales"].median():.2f}')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Statistics: Mean={df_sample['unit_sales'].mean():.2f}, Median={df_sample['unit_sales'].median():.2f}, Std={df_sample['unit_sales'].std():.2f}")

### Visualization 3: Average Sales by Product Family

In [ ]:
plt.figure(figsize=(14, 8))

family_sales = df_sample.groupby('family')['unit_sales'].mean().sort_values(ascending=True)
family_sales.plot(kind='barh', color='teal', edgecolor='black')
plt.title('Average Sales by Product Family', fontsize=16, fontweight='bold')
plt.xlabel('Average Unit Sales (log scale)', fontsize=12)
plt.ylabel('Product Family', fontsize=12)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print(f"Top 5 families by average sales:")
print(family_sales.tail())

### Visualization 4: Promotion Impact (On vs Off)

In [ ]:
plt.figure(figsize=(10, 6))

df_sample['promo_status'] = df_sample['onpromotion'].map({0: 'No Promotion', 1: 'On Promotion'})
sns.boxplot(data=df_sample, x='promo_status', y='unit_sales', palette='Set2')
plt.title('Promotion Impact on Unit Sales', fontsize=16, fontweight='bold')
plt.xlabel('Promotion Status', fontsize=12)
plt.ylabel('Unit Sales (log scale)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

promo_stats = df_sample.groupby('onpromotion')['unit_sales'].agg(['mean', 'median', 'count'])
print("\nPromotion Statistics:")
print(promo_stats)

### Visualization 5: Day-of-Week Seasonality

In [ ]:
plt.figure(figsize=(12, 6))

day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_sales = df_sample.groupby('day_of_week')['unit_sales'].mean()
plt.bar(range(7), dow_sales.values, color='skyblue', edgecolor='black')
plt.xticks(range(7), day_names, rotation=45)
plt.title('Average Sales by Day of Week', fontsize=16, fontweight='bold')
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Average Unit Sales (log scale)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nDay of Week Statistics:")
print(dow_sales)

### Visualization 6: Autocorrelation Plot (Lags 1-30)

In [ ]:
from pandas.plotting import autocorrelation_plot

plt.figure(figsize=(12, 6))

# Sample a single store-item combination for autocorrelation
sample_series = df_sample[(df_sample['store_nbr'] == df_sample['store_nbr'].iloc[0]) & 
                   (df_sample['item_nbr'] == df_sample['item_nbr'].iloc[0])].sort_values('date')['unit_sales']

autocorrelation_plot(sample_series)
plt.title('Autocorrelation Plot (Sample Store-Item)', fontsize=16, fontweight='bold')
plt.xlabel('Lag', fontsize=12)
plt.ylabel('Autocorrelation', fontsize=12)
plt.xlim(0, 30)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Key lags show strong correlation at 1, 7, 14 days (weekly patterns)")

### Visualization 7: Rolling Mean Illustration

In [ ]:
plt.figure(figsize=(15, 6))

# Select one store-item for visualization
visualization_df = df_sample[(df_sample['store_nbr'] == df_sample['store_nbr'].iloc[0]) & 
               (df_sample['item_nbr'] == df_sample['item_nbr'].iloc[0])].sort_values('date').head(90)

plt.plot(visualization_df['date'], visualization_df['unit_sales'], label='Actual Sales', alpha=0.6, linewidth=1)
plt.plot(visualization_df['date'], visualization_df['rolling_mean_7'], label='7-Day MA', linewidth=2)
plt.plot(visualization_df['date'], visualization_df['rolling_mean_14'], label='14-Day MA', linewidth=2)

plt.title('Actual Sales vs Rolling Means (Sample Store-Item, 90 days)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Unit Sales (log scale)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Rolling means smooth out daily volatility and capture trends")

### Visualization 8: Holiday Effect

In [ ]:
plt.figure(figsize=(10, 6))

df_sample['holiday_status'] = df_sample['is_holiday'].map({0: 'Regular Day', 1: 'Holiday'})
sns.boxplot(data=df_sample, x='holiday_status', y='unit_sales', palette='Set1')
plt.title('Holiday vs Non-Holiday Sales', fontsize=16, fontweight='bold')
plt.xlabel('Day Type', fontsize=12)
plt.ylabel('Unit Sales (log scale)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

holiday_stats = df_sample.groupby('is_holiday')['unit_sales'].agg(['mean', 'median', 'count'])
print("\nHoliday Statistics:")
print(holiday_stats)

### Visualization 9: Perishable vs Non-Perishable Patterns

In [ ]:
plt.figure(figsize=(14, 6))

perishable_daily = df_sample.groupby(['date', 'perishable'])['unit_sales'].mean().reset_index()

for p in [0, 1]:
    data = perishable_daily[perishable_daily['perishable'] == p]
    label = 'Non-Perishable' if p == 0 else 'Perishable'
    plt.plot(data['date'], data['unit_sales'], label=label, alpha=0.7, linewidth=1)

plt.title('Perishable vs Non-Perishable Sales Patterns', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Average Unit Sales (log scale)', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

perishable_stats = df_sample.groupby('perishable')['unit_sales'].agg(['mean', 'std', 'count'])
print("\nPerishable vs Non-Perishable Statistics:")
print(perishable_stats)

### Visualization 10: Feature Correlation Heatmap (Top 20 Features)

In [ ]:
plt.figure(figsize=(16, 12))

# Select numeric features for correlation
numeric_features = ['unit_sales'] + all_engineered_features + ['onpromotion', 'perishable', 
                                                                'dcoilwtico', 'transactions']
numeric_features = [f for f in numeric_features if f in df_sample.columns]

# Calculate correlation with unit_sales
correlations = df_sample[numeric_features].corr()['unit_sales'].abs().sort_values(ascending=False)

# Top 20 features
top_20_features = correlations.head(20).index.tolist()
corr_matrix = df_sample[top_20_features].corr()

# Heatmap
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap (Top 20 Features)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTop 10 Features Correlated with Unit Sales:")
print(correlations.head(10))

## 7. Train/Test Split (Time-Based on FULL Dataset)

In [ ]:
print("="*70)
print("TRAIN/TEST SPLIT ON FULL DATASET")
print("="*70)
print("\nUsing FULL dataset (not sample) for final train/test split")
print(f"Full dataset shape: {df.shape}\n")

# Define split dates
train_end_date = '2017-08-15'
test_start_date = '2017-08-16'
test_end_date = '2017-08-31'

# Split the FULL data
df_train = df[df['date'] <= train_end_date].copy()
df_test = df[(df['date'] >= test_start_date) & (df['date'] <= test_end_date)].copy()

print(f"✓ Train Set: {df_train.shape}")
print(f"  Date range: {df_train['date'].min()} to {df_train['date'].max()}")

print(f"\n✓ Test Set: {df_test.shape}")
print(f"  Date range: {df_test['date'].min()} to {df_test['date'].max()}")

print(f"\nTest set represents {len(df_test) / len(df) * 100:.2f}% of total data")
print(f"Covers {(df_test['date'].max() - df_test['date'].min()).days + 1} days (16 days for forecasting)")

# Save train and test sets
print("\n" + "="*70)
print("SAVING TRAIN/TEST DATASETS")
print("="*70)

train_path = '../results/df_train.csv'
test_path = '../results/df_test.csv'

df_train.to_csv(train_path, index=False)
df_test.to_csv(test_path, index=False)

print(f"✓ Training set saved to: {train_path}")
print(f"  Shape: {df_train.shape}")
print(f"✓ Test set saved to: {test_path}")
print(f"  Shape: {df_test.shape}")

## 8. Final Dataset Summary

In [ ]:
print("="*70)
print("FINAL DATASET SUMMARY - READY FOR MODELING")
print("="*70)

print(f"\n📊 FULL Dataset Shape: {df.shape}")
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")

print(f"\n📅 Date Range:")
print(f"  Start: {df['date'].min()}")
print(f"  End: {df['date'].max()}")
print(f"  Duration: {(df['date'].max() - df['date'].min()).days} days")

print(f"\n🏪 Coverage:")
print(f"  Unique stores: {df['store_nbr'].nunique()}")
print(f"  Unique items: {df['item_nbr'].nunique()}")
print(f"  Product families: {df['family'].nunique()}")

print(f"\n🎯 Target Variable:")
print(f"  Name: unit_sales (log-transformed)")
print(f"  Mean: {df['unit_sales'].mean():.4f}")
print(f"  Std: {df['unit_sales'].std():.4f}")
print(f"  Missing: {df['unit_sales'].isna().sum()}")

print(f"\n🔧 Engineered Features: {len(all_engineered_features)}")
print(f"  Temporal: {len(temporal_features)}")
print(f"  Lag: {len(lag_features)}")
print(f"  Rolling: {len(rolling_features)}")
print(f"  Holiday: {len(holiday_features)}")
print(f"  Promotion: {len(promo_features)}")
print(f"  Aggregate: {len(aggregate_features)}")
print(f"  Interaction: {len(interaction_features)}")

print(f"\n📋 All Columns ({len(df.columns)}):")
print(list(df.columns))

print(f"\n📈 Missing Values Summary:")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print(missing.head(10))
else:
    print("  No missing values in primary features")

print("\n" + "="*70)
print("SAVED FILES SUMMARY")
print("="*70)
print("\n📁 CSV Files Created:")
print("  1. ../results/df_merged.csv - Merged dataset (125M rows)")
print("  2. ../results/df_featured_full.csv - Full featured dataset (125M rows)")
print("  3. ../results/df_sample_2m.csv - Sample for visualizations (2M rows)")
print("  4. ../results/df_train.csv - Training set (2013-01-01 to 2017-08-15)")
print("  5. ../results/df_test.csv - Test set (2017-08-16 to 2017-08-31)")

print("\n✅ Data is ready for XGBoost/LightGBM modeling!")
print("✅ Goal: Predict unit_sales 1-16 days ahead for each (store, item, date)")
print("✅ Metric: RMSLE (Root Mean Squared Logarithmic Error)")
print("="*70)